<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/treinar_cerebro_triplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =======================================================================================
# PARCEIRO DE PROGRAMAÇÃO: SCRIPT DE TREINO DO "CÉREBRO TRIPLO" (CNN + LSTM + TRANSFORMER)
# Autor: Rondinelli Alves de Andrade.
# =======================================================================================
print("🚀 Iniciando o processo de treino do 'Cérebro Triplo'...")

# --- Parte 1: Importações ---
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from datetime import date
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, LSTM, concatenate, GlobalAveragePooling1D, LayerNormalization, MultiHeadAttention
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping
import joblib

print("✅ Bibliotecas carregadas.")

# --- Parte 2: Configurações ---
TICKER = 'ETH-USD'  # Ativo para treinar
START_DATE = '2017-01-01' # Data de início da coleta de dados
END_DATE = date.today().strftime("%Y-%m-%d")
SEQ_LENGTH = 60
FUTURE_DAYS = 5
THRESHOLD = 0.04  # 4% de variação para definir ALTA/BAIXA (ajustável)
IMG_SIZE = 128
BASE_DIR = 'imagens_treino_triplo' # Pasta temporária para as imagens

# Parâmetros do Transformer
HEAD_SIZE = 256
NUM_HEADS = 4
FF_DIM = 4
NUM_BLOCKS = 4
DROPOUT_RATE = 0.2

# Parâmetros de Treinamento
BATCH_SIZE = 32
EPOCHS = 60 # Pode precisar de mais épocas por ser um modelo complexo

if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)
os.makedirs(BASE_DIR, exist_ok=True)
print(f"✅ Configurações definidas para treinar o ativo: {TICKER}")


# --- Parte 3: Obtenção e Preparação de Dados ---
# (Esta parte é idêntica à do Cérebro Híbrido)
print(f"📥 Baixando dados para {TICKER}...")
data = yf.download(TICKER, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
data.dropna(inplace=True)

print("📈 Calculando indicadores técnicos...")
window = 20
data['volatility_bbm'] = data['Close'].rolling(window=window).mean()
data['volatility_bbh'] = data['Close'].rolling(window=window).mean() + data['Close'].rolling(window=window).std() * 2
data['volatility_bbl'] = data['Close'].rolling(window=window).mean() - data['Close'].rolling(window=window).std() * 2
window_rsi = 14
delta = data['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.ewm(com=window_rsi - 1, adjust=False).mean()
avg_loss = loss.ewm(com=window_rsi - 1, adjust=False).mean()
rs = avg_gain / avg_loss
rs = rs.replace([np.inf, -np.inf], np.nan).fillna(0)
data['momentum_rsi'] = 100 - (100 / (1 + rs))
data.dropna(inplace=True)
print("✅ Indicadores calculados.")


# --- Parte 4: Geração das Imagens e Sequências ---
# (Esta parte também é idêntica à do Cérebro Híbrido)
print("🧩 Gerando imagens e sequências para o treino...")
features_numerical = ['Close', 'Volume', 'volatility_bbh', 'volatility_bbl', 'volatility_bbm', 'momentum_rsi']
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = data.copy()
data_normalized[features_numerical] = scaler.fit_transform(data[features_numerical])

image_paths, numerical_sequences, labels = [], [], []
class_names = ['ALTA', 'BAIXA', 'NEUTRA']
class_map = {name: i for i, name in enumerate(class_names)}

for i in range(len(data) - SEQ_LENGTH - FUTURE_DAYS):
    preco_final_janela = data['Close'].iloc[i + SEQ_LENGTH - 1]
    preco_futuro = data['Close'].iloc[i + SEQ_LENGTH + FUTURE_DAYS - 1]

    label_str = 'NEUTRA'
    if preco_futuro.iloc[0] > preco_final_janela.iloc[0] * (1 + THRESHOLD):
        label_str = 'ALTA'
    elif preco_futuro.iloc[0] < preco_final_janela.iloc[0] * (1 - THRESHOLD):
        label_str = 'BAIXA'
    labels.append(class_map[label_str])

    dados_janela = data_normalized.iloc[i : i + SEQ_LENGTH]
    caminho_completo = os.path.join(BASE_DIR, f"img_{i}.png")

    fig, ax = plt.subplots(3, 1, figsize=(IMG_SIZE/100, IMG_SIZE/100), dpi=100, gridspec_kw={'height_ratios': [2, 1, 1]})
    ax[0].plot(dados_janela['Close'].values, color='black'); ax[0].plot(dados_janela['volatility_bbh'].values, color='blue', alpha=0.5); ax[0].plot(dados_janela['volatility_bbl'].values, color='blue', alpha=0.5)
    ax[1].plot(dados_janela['momentum_rsi'].values, color='purple')
    ax[2].bar(range(len(dados_janela)), dados_janela['Volume'].values.flatten(), color='gray')
    for axis in ax: axis.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0.01)
    fig.savefig(caminho_completo, bbox_inches='tight', pad_inches=0)
    plt.close(fig)

    image_paths.append(caminho_completo)
    numerical_sequences.append(dados_janela[features_numerical].values)

labels = np.array(labels, dtype=np.int32)
img_train_paths, img_val_paths, X_train_num, X_val_num, y_train, y_val = train_test_split(
    np.array(image_paths), np.array(numerical_sequences), labels, test_size=0.2, random_state=42, stratify=labels
)
print(f"✅ {len(labels)} exemplos gerados e divididos.")


# --- Parte 5: Construção do Modelo "Cérebro Triplo" ---
print("\n🤖 Construindo a arquitetura do 'Cérebro Triplo'...")

# --- Bloco de construção do Transformer (o nosso terceiro especialista) ---
def transformer_encoder(inputs):
    attention = MultiHeadAttention(key_dim=HEAD_SIZE, num_heads=NUM_HEADS, dropout=DROPOUT_RATE)(inputs, inputs)
    attention = Dropout(DROPOUT_RATE)(attention)
    attention = LayerNormalization(epsilon=1e-6)(inputs + attention)
    ffn = keras.Sequential([Dense(FF_DIM, activation="relu"), Dense(inputs.shape[-1])])
    outputs = ffn(attention)
    outputs = Dropout(DROPOUT_RATE)(outputs)
    outputs = LayerNormalization(epsilon=1e-6)(attention + outputs)
    return outputs

# --- Entradas do Modelo (uma para imagem, uma para dados numéricos) ---
input_cnn = Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='entrada_imagem')
input_numerical = Input(shape=(SEQ_LENGTH, len(features_numerical)), name='entrada_numerica')

# --- Ramo 1: Especialista CNN (O Analista Visual) ---
cnn_branch = Rescaling(1./255)(input_cnn)
cnn_branch = Conv2D(32, 3, activation='relu')(cnn_branch)
cnn_branch = MaxPooling2D()(cnn_branch)
cnn_branch = Conv2D(64, 3, activation='relu')(cnn_branch)
cnn_branch = MaxPooling2D()(cnn_branch)
cnn_branch = Flatten()(cnn_branch)
cnn_output = Dense(64, activation='relu', name='saida_cnn')(cnn_branch)

# --- Ramo 2: Especialista LSTM (O Analista de Memória Curta) ---
lstm_branch = LSTM(units=50)(input_numerical)
lstm_branch = Dropout(0.2)(lstm_branch)
lstm_output = Dense(25, activation='relu', name='saida_lstm')(lstm_branch)

# --- Ramo 3: Especialista Transformer (O Analista de Contexto) ---
# A entrada numérica é partilhada com o ramo Transformer
transformer_branch = input_numerical
for _ in range(NUM_BLOCKS):
    transformer_branch = transformer_encoder(transformer_branch)
transformer_branch = GlobalAveragePooling1D(data_format="channels_last")(transformer_branch)
transformer_output = Dense(64, activation='relu', name='saida_transformer')(transformer_branch)

# --- A "Mesa Redonda": Unindo as opiniões dos três especialistas ---
fused = concatenate([cnn_output, lstm_output, transformer_output])

# --- A Cabeça de Decisão Final ---
final_head = Dense(64, activation='relu')(fused)
final_head = Dropout(0.5)(final_head)
final_output = Dense(len(class_names), activation='softmax', name='saida_final')(final_head)

# --- Criação do Modelo Final ---
trifecta_model = Model(
    inputs=[input_cnn, input_numerical],
    outputs=final_output,
    name="Cerebro_Triplo"
)
trifecta_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
trifecta_model.summary()
print("✅ Arquitetura do 'Cérebro Triplo' construída!")


# --- Parte 6: Treinamento ---
# O gerador de dados agora alimenta as duas entradas do nosso modelo
def data_generator(img_paths, numerical_seqs, lbls, b_size):
    num_samples = len(img_paths)
    while True:
        indices = np.arange(num_samples); np.random.shuffle(indices)
        for offset in range(0, num_samples, b_size):
            batch_indices = indices[offset:offset+b_size]
            batch_images = np.array([img_to_array(load_img(p, target_size=(IMG_SIZE, IMG_SIZE))) for p in img_paths[batch_indices]])
            batch_numerical = numerical_seqs[batch_indices]

            # Yield um dicionário com as duas entradas
            yield ({'entrada_imagem': batch_images, 'entrada_numerica': batch_numerical}, lbls[batch_indices])

train_gen = data_generator(img_train_paths, X_train_num, y_train, BATCH_SIZE)
val_gen = data_generator(img_val_paths, X_val_num, y_val, BATCH_SIZE)

print("\n🧠 Iniciando o treinamento do 'Cérebro Triplo'. Este passo será demorado...")
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

trifecta_model.fit(
    train_gen,
    steps_per_epoch=len(y_train) // BATCH_SIZE,
    validation_data=val_gen,
    validation_steps=len(y_val) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)
print("✅ Treinamento concluído.")

# --- Parte 7: Salvar os Artefactos ---
print(f"\n💾 Salvando os artefactos do modelo para o ativo {TICKER}...")
# Renomeia os ficheiros de saída para incluir o nome do ticker, prontos para produção
model_save_path = f'analista_triplo_{TICKER}.keras'
scaler_save_path = f'scaler_triplo_{TICKER}.gz'

trifecta_model.save(model_save_path)
joblib.dump(scaler, scaler_save_path)
shutil.rmtree(BASE_DIR) # Limpa a pasta de imagens temporárias

print("\n>>> PROCESSO DE TREINO E SALVAMENTO CONCLUÍDO! <<<")
print(f"✓ Ficheiro do modelo salvo em: {model_save_path}")
print(f"✓ Ficheiro do scaler salvo em: {scaler_save_path}")

🚀 Iniciando o processo de treino do 'Cérebro Triplo'...
✅ Bibliotecas carregadas.
✅ Configurações definidas para treinar o ativo: ETH-USD
📥 Baixando dados para ETH-USD...
📈 Calculando indicadores técnicos...
✅ Indicadores calculados.
🧩 Gerando imagens e sequências para o treino...
✅ 2806 exemplos gerados e divididos.

🤖 Construindo a arquitetura do 'Cérebro Triplo'...


Model: "Cerebro_Triplo"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ entrada_numerica    │ (None, 60, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 6)     │     27,654 │ entrada_numerica… │
│ (MultiHeadAttentio… │                   │            │ entrada_numerica… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 60, 6)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 60, 6)     │          0 │ entrada_numerica… │
│                     │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 60, 6)     │         12 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 60, 6)     │         58 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 60, 6)     │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 60, 6)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 6)     │         12 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 6)     │     27,654 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 60, 6)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 60, 6)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 6)     │         12 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 60, 6)     │         58 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 60, 6)     │          0 │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 60, 6)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 60, 6)     │         12 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 60, 6)     │     27,654 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat

 Total params: 3,839,974 (14.65 MB)

 Trainable params: 3,839,974 (14.65 MB)

 Non-trainable params: 0 (0.00 B)

✅ Arquitetura do 'Cérebro Triplo' construída!

🧠 Iniciando o treinamento do 'Cérebro Triplo'. Este passo será demorado...
Epoch 1/60
70/70 ━━━━━━━━━━━━━━━━━━━━ 22s 76ms/step - accuracy: 0.3660 - loss: 1.3371 - val_accuracy: 0.4099 - val_loss: 1.0855
Epoch 2/60
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.3858 - loss: 1.0871 - val_accuracy: 0.4724 - val_loss: 1.0525
Epoch 3/60
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.4769 - loss: 1.0329 - val_accuracy: 0.5037 - val_loss: 0.9875
Epoch 4/60
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - accuracy: 0.5823 - loss: 0.9100 - val_accuracy: 0.5170 - val_loss: 0.9561
Epoch 5/60
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.6483 - loss: 0.7861 - val_accuracy: 0.5919 - val_loss: 0.8866
Epoch 6/60
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 63ms/step - accuracy: 0.7509 - loss: 0.6331 - val_accuracy: 0.6472 - val_loss: 0.8092
Epoch 7/60
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.8217 - loss: 0.4584 - val_accuracy: 0.6717 - val_l